In [ ]:
import os
from helpers.google import (
    grounded_generation,
)

from google import genai

G_API_KEY = os.environ.get("GOOGLE_API_KEY")

client = genai.Client(api_key=G_API_KEY)

In [252]:
def grounding_chunks_to_footnotes(grounding_chunks):
    """
    Create markdown footnote list from Google Grounded Search "grounding chunks".

    Parameters
    ----------
    grounding_chunks : list[google.genai.types.GroundingChunk]
        A list of grounding chunk objects from Google Grounded search where each
        object has 'web.title' and 'web.uri' attributes.

    Returns
    -------
    str
        Markdown footnote list, numbered sequentially, with URIs hyperlinked.

    Examples
    --------
    >>> grounding_chunks_to_footnote_list(grounding_chunks)
    '1. [Title 1](http://example.com/1)\n2. [Title 2](http://example.com/2)\n'
    """
    grounding_chunks_str = ""
    for idx, chunk in enumerate(grounding_chunks, start=1):
        grounding_chunks_str += f"{idx}. [{chunk.web.title}]({chunk.web.uri})\n"
    return grounding_chunks_str


def update_response_text(full_text, grounding_supports, grounding_chunks):
    """
    Update the response text with markdown footnotes.

    Parameters
    ----------
    full_text : str
        The full text of the API response.
    grounding_supports : list[google.genai.types.GroundingSupport]
        A list of grounding support objects from Google Grounded search.
    grounding_chunks : list[google.genai.types.GroundingChunk]
        A list of grounding chunk objects from Google Grounded search.

    Returns
    -------
    str
        The updated response text with markdown footnotes.
    """

    updated_text = full_text

    offset = 0
    for support in grounding_supports:
        support_end_idx = support.segment.end_index + offset

        chunk_indices = support.grounding_chunk_indices

        ground_dicts = []

        for chunk_index in chunk_indices:
            chunk = grounding_chunks[chunk_index]
            ground_dicts.append(
                {
                    "cite_index": chunk_index + 1,
                    "link": chunk.web.uri,
                }
            )

        # Update the text
        prefix = updated_text[:support_end_idx]
        suffix = updated_text[support_end_idx:]

        inserted_text = " ["  # Open the citation
        for ground_dict in ground_dicts:

            # Each citation number links to the source
            cite_index = ground_dict["cite_index"]
            link = ground_dict["link"]
            inserted_text += f"[{cite_index}]({link}), "

        inserted_text = inserted_text.rstrip(", ")  # Remove trailing comma
        inserted_text += f"]"  # Close the citation

        # Update the offset for the next insertion
        offset += len(inserted_text)

        updated_text = f"{prefix}{inserted_text}{suffix}"

    updated_text += f"\n\n"
    footnotes = grounding_chunks_to_footnotes(grounding_chunks)
    updated_text += footnotes

    return updated_text

In [110]:
# Ask who the current US president is
question = "what is the best way to clean a wound?"
response = grounded_generation(client, question)

In [253]:
full_text = response.candidates[0].content.parts[0].text

full_text

"The best way to clean a wound involves several steps:\n\n1.  **Wash Your Hands:** Before you even touch the wound, make sure to wash your hands thoroughly with soap and water or use a hand sanitizer. If possible, wear disposable gloves.\n\n2.  **Stop the Bleeding:** If the wound is bleeding, apply gentle pressure using a clean cloth.\n\n3.  **Rinse the Wound:** Use clear, running water to rinse the wound. This helps to loosen and remove dirt and debris. You can also use a sterile saline solution to irrigate the wound. A syringe can be used to create gentle pressure to help remove debris.\n\n4.  **Clean Around the Wound:** Use a soft washcloth and mild soap to gently clean the skin around the wound. Avoid getting soap directly in the wound, as it can cause irritation.\n\n5.  **Remove Debris:** If there is any dirt or debris remaining in the wound after washing, use tweezers to remove it. Make sure to clean the tweezers first with isopropyl alcohol. If you can't remove all debris, it's 

In [256]:
print(
    update_response_text(
        full_text,
        response.candidates[0].grounding_metadata.grounding_supports,
        response.candidates[0].grounding_metadata.grounding_chunks,
    )
)

The best way to clean a wound involves several steps:

1.  **Wash Your Hands:** Before you even touch the wound, make sure to wash your hands thoroughly with soap and water or use a hand sanitizer. If possible, wear disposable gloves. [[1](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYs7nLJ_sEz4uZ3XveeCDOUcpcgvpGJuu1a75zSwZFB2kfYkDOCXn4aEWHjWoUZY7yR5xud9m-5dMSrVWJOp2vWpBQib6Zpw0gBn9FRGxVGXXPqQgo-ae2sVgFTgbz-8itMZPo-VSQd8eovWcEjLKTI=)]

2.  **Stop the Bleeding:** If the wound is bleeding, apply gentle pressure using a clean cloth.

3.  **Rinse the Wound:** Use clear, running water to rinse the wound. This helps to loosen and remove dirt and debris. You can also use a sterile saline solution to irrigate the wound. [[2](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYu1LFk0f4bw_7QXkMGxgbzWg_zisfbonQapThnW7JvEU5zJb9wy5NuCaBOukFNVyHGedEZy7Zz1EZQXhxKZfb6xh1_Z5Npc6wQLPiBmH2Io3HoJREIUZz4FxRgsvwsIfD9aSbw3fKJrS7sr0eZoHbP3jmTNF8XeKMCt-tEhndlM1NgmxhPPCTl

In [143]:
response.candidates[0].grounding_metadata.grounding_chunks

[GroundingChunk(retrieved_context=None, web=GroundingChunkWeb(title='webmd.com', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYs7nLJ_sEz4uZ3XveeCDOUcpcgvpGJuu1a75zSwZFB2kfYkDOCXn4aEWHjWoUZY7yR5xud9m-5dMSrVWJOp2vWpBQib6Zpw0gBn9FRGxVGXXPqQgo-ae2sVgFTgbz-8itMZPo-VSQd8eovWcEjLKTI=')),
 GroundingChunk(retrieved_context=None, web=GroundingChunkWeb(title='msdmanuals.com', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYu1LFk0f4bw_7QXkMGxgbzWg_zisfbonQapThnW7JvEU5zJb9wy5NuCaBOukFNVyHGedEZy7Zz1EZQXhxKZfb6xh1_Z5Npc6wQLPiBmH2Io3HoJREIUZz4FxRgsvwsIfD9aSbw3fKJrS7sr0eZoHbP3jmTNF8XeKMCt-tEhndlM1NgmxhPPCTlBLqLELeoUGXsLSNXXz62qjGkYgnOW8CwtZhwi49Q27INtk2clXbhu0hsWhai3in91V5VAvar2yhcI6DqP2a9AWjI41Gl4NQ==')),
 GroundingChunk(retrieved_context=None, web=GroundingChunkWeb(title='healthxchange.sg', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYv4ogWt6yP5MQJgQzQY_i6k3hmzPhgwfsDDv5PH7iFlWtTFOxdJ3QoLzt9iGyc1oEwQX89e4AYT46tzeLtFl6l

In [144]:
response.candidates[0].grounding_metadata.grounding_supports

[GroundingSupport(confidence_scores=[0.9405256], grounding_chunk_indices=[0], segment=Segment(end_index=234, part_index=None, start_index=198, text='If possible, wear disposable gloves.')),
 GroundingSupport(confidence_scores=[0.8517451], grounding_chunk_indices=[1], segment=Segment(end_index=517, part_index=None, start_index=452, text='You can also use a sterile saline solution to irrigate the wound.')),
 GroundingSupport(confidence_scores=[0.97488904], grounding_chunk_indices=[2], segment=Segment(end_index=588, part_index=None, start_index=518, text='A syringe can be used to create gentle pressure to help remove debris.')),
 GroundingSupport(confidence_scores=[0.9142004, 0.8503812], grounding_chunk_indices=[0, 3], segment=Segment(end_index=769, part_index=None, start_index=700, text='Avoid getting soap directly in the wound, as it can cause irritation.')),
 GroundingSupport(confidence_scores=[0.84713405, 0.81815046], grounding_chunk_indices=[0, 3], segment=Segment(end_index=889, part

In [118]:
support.grounding_chunk_indices

[0]

In [153]:
grounding_supports = response.candidates[0].grounding_metadata.grounding_supports
grounding_chunks = response.candidates[0].grounding_metadata.grounding_chunks

In [154]:
len(grounding_supports)

9

In [147]:
print(full_text)

The best way to clean a wound involves several steps:

1.  **Wash Your Hands:** Before you even touch the wound, make sure to wash your hands thoroughly with soap and water or use a hand sanitizer. If possible, wear disposable gloves.

2.  **Stop the Bleeding:** If the wound is bleeding, apply gentle pressure using a clean cloth.

3.  **Rinse the Wound:** Use clear, running water to rinse the wound. This helps to loosen and remove dirt and debris. You can also use a sterile saline solution to irrigate the wound. A syringe can be used to create gentle pressure to help remove debris.

4.  **Clean Around the Wound:** Use a soft washcloth and mild soap to gently clean the skin around the wound. Avoid getting soap directly in the wound, as it can cause irritation.

5.  **Remove Debris:** If there is any dirt or debris remaining in the wound after washing, use tweezers to remove it. Make sure to clean the tweezers first with isopropyl alcohol. If you can't remove all debris, it's best to see

In [251]:
def update_response_text(full_text, grounding_supports, grounding_chunks):
    """
    Update the response text with markdown footnotes.

    Parameters
    ----------
    full_text : str
        The full text of the API response.
    grounding_supports : list[google.genai.types.GroundingSupport]
        A list of grounding support objects from Google Grounded search.
    grounding_chunks : list[google.genai.types.GroundingChunk]
        A list of grounding chunk objects from Google Grounded search.

    Returns
    -------
    str
        The updated response text with markdown footnotes.
    """

    updated_text = full_text

    offset = 0
    for support in grounding_supports:
        support_end_idx = support.segment.end_index + offset

        chunk_indices = support.grounding_chunk_indices

        ground_dicts = []

        for chunk_index in chunk_indices:
            chunk = grounding_chunks[chunk_index]
            ground_dicts.append(
                {
                    "cite_index": chunk_index + 1,
                    "link": chunk.web.uri,
                }
            )

        # Split the prefix and suffix, accounting for new line chars
        prefix = updated_text[:support_end_idx]
        suffix = updated_text[support_end_idx:]
        newline_str = collect_trailing_newlines(prefix)
        prefix = prefix.rstrip(newline_str)

        # Update the text
        inserted_text = " ["  # Open the citation
        for ground_dict in ground_dicts:
            # Each citation number includes it's own link
            cite_index = ground_dict["cite_index"]
            link = ground_dict["link"]
            inserted_text += f"[{cite_index}]({link}), "
        inserted_text = inserted_text.rstrip(", ")  # Remove trailing comma
        inserted_text += f"]"  # Close the citation

        # Update the offset for the next insertion
        offset += len(inserted_text)

        updated_text = f"{prefix}{inserted_text}{suffix}"

    updated_text += f"\n\n"
    footnotes = grounding_chunks_to_footnotes(grounding_chunks)
    updated_text += footnotes

    return updated_text


updated_text = update_response_text(full_text, grounding_supports, grounding_chunks)
print(updated_text)

The best way to clean a wound involves several steps:

1.  **Wash Your Hands:** Before you even touch the wound, make sure to wash your hands thoroughly with soap and water or use a hand sanitizer. If possible, wear disposable gloves. [[1](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYs7nLJ_sEz4uZ3XveeCDOUcpcgvpGJuu1a75zSwZFB2kfYkDOCXn4aEWHjWoUZY7yR5xud9m-5dMSrVWJOp2vWpBQib6Zpw0gBn9FRGxVGXXPqQgo-ae2sVgFTgbz-8itMZPo-VSQd8eovWcEjLKTI=)]

2.  **Stop the Bleeding:** If the wound is bleeding, apply gentle pressure using a clean cloth.

3.  **Rinse the Wound:** Use clear, running water to rinse the wound. This helps to loosen and remove dirt and debris. You can also use a sterile saline solution to irrigate the wound. [[2](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYu1LFk0f4bw_7QXkMGxgbzWg_zisfbonQapThnW7JvEU5zJb9wy5NuCaBOukFNVyHGedEZy7Zz1EZQXhxKZfb6xh1_Z5Npc6wQLPiBmH2Io3HoJREIUZz4FxRgsvwsIfD9aSbw3fKJrS7sr0eZoHbP3jmTNF8XeKMCt-tEhndlM1NgmxhPPCTl

In [234]:
updated_text = full_text

offset = 0
for support in grounding_supports:
    support_end_idx = support.segment.end_index + offset

    chunk_indices = support.grounding_chunk_indices

    ground_dicts = []

    for chunk_index in chunk_indices:
        chunk = grounding_chunks[chunk_index]
        ground_dicts.append(
            {
                "cite_index": chunk_index + 1,
                "link": chunk.web.uri,
                "source": chunk.web.title,
            }
        )

    # Split the prefix and suffix, accounting for new line chars
    prefix = updated_text[:support_end_idx]
    suffix = updated_text[support_end_idx:]
    newline_str = collect_trailing_newlines(prefix)
    prefix = prefix.rstrip(newline_str)

    # Update the text
    inserted_text = "["  # Open the citation
    for ground_dict in ground_dicts:
        # Each citation number includes it's own link
        cite_index = ground_dict["cite_index"]
        link = ground_dict["link"]
        source = ground_dict["source"]
        inserted_text += f"[^{cite_index}]({link}), "
    inserted_text = inserted_text.rstrip(", ")  # Remove trailing comma
    inserted_text += f"]"  # Close the citation

    # Update the offset for the next insertion
    offset += len(inserted_text)

    updated_text = f"{prefix}{inserted_text}{suffix}"

In [240]:
grounding_chunks[0]

GroundingChunk(retrieved_context=None, web=GroundingChunkWeb(title='webmd.com', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYs7nLJ_sEz4uZ3XveeCDOUcpcgvpGJuu1a75zSwZFB2kfYkDOCXn4aEWHjWoUZY7yR5xud9m-5dMSrVWJOp2vWpBQib6Zpw0gBn9FRGxVGXXPqQgo-ae2sVgFTgbz-8itMZPo-VSQd8eovWcEjLKTI='))

In [243]:
def grounding_chunks_to_citation_list(grounding_chunks):
    """
    Create markdown citation list with source and link.

    Parameters
    ----------
    grounding_chunks : list[google.genai.types.GroundingChunk]
        A list of grounding chunk objects from Google Grounded search where each
        object has 'web.title' and 'web.uri' attributes.

    Returns
    -------
    str
        Markdown citation list, numbered sequentially, with URIs hyperlinked.

    Examples
    --------
    >>> grounding_chunks_to_citation_list(grounding_chunks)
    '1. [Title 1](http://example.com/1)\n2. [Title 2](http://example.com/2)\n'
    """
    grounding_chunks_str = ""
    for idx, chunk in enumerate(grounding_chunks, start=1):
        grounding_chunks_str += f"{idx}. [{chunk.web.title}]({chunk.web.uri})\n"
    return grounding_chunks_str


# Example usage
grounding_chunks_str = grounding_chunks_to_citation_list(grounding_chunks)
print(grounding_chunks_str)

1. [webmd.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYs7nLJ_sEz4uZ3XveeCDOUcpcgvpGJuu1a75zSwZFB2kfYkDOCXn4aEWHjWoUZY7yR5xud9m-5dMSrVWJOp2vWpBQib6Zpw0gBn9FRGxVGXXPqQgo-ae2sVgFTgbz-8itMZPo-VSQd8eovWcEjLKTI=)
2. [msdmanuals.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYu1LFk0f4bw_7QXkMGxgbzWg_zisfbonQapThnW7JvEU5zJb9wy5NuCaBOukFNVyHGedEZy7Zz1EZQXhxKZfb6xh1_Z5Npc6wQLPiBmH2Io3HoJREIUZz4FxRgsvwsIfD9aSbw3fKJrS7sr0eZoHbP3jmTNF8XeKMCt-tEhndlM1NgmxhPPCTlBLqLELeoUGXsLSNXXz62qjGkYgnOW8CwtZhwi49Q27INtk2clXbhu0hsWhai3in91V5VAvar2yhcI6DqP2a9AWjI41Gl4NQ==)
3. [healthxchange.sg](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYv4ogWt6yP5MQJgQzQY_i6k3hmzPhgwfsDDv5PH7iFlWtTFOxdJ3QoLzt9iGyc1oEwQX89e4AYT46tzeLtFl6lVu7wvlOKlLshUdlD-Fzi31iwRnfJayp1M71PkV-dqGuDMy-D-TuWvpjJBDfiUR2A1rCGH0Ts8sxaBKAaj1bdIhnFbQa5KTnxJhfKwqZwxBw==)
4. [mayoclinic.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYu4hhFHun-6NFEdWHw7L

In [245]:
type(grounding_chunks[0])

google.genai.types.GroundingChunk

In [235]:
print(updated_text)

The best way to clean a wound involves several steps:

1.  **Wash Your Hands:** Before you even touch the wound, make sure to wash your hands thoroughly with soap and water or use a hand sanitizer. If possible, wear disposable gloves.[[^1](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYs7nLJ_sEz4uZ3XveeCDOUcpcgvpGJuu1a75zSwZFB2kfYkDOCXn4aEWHjWoUZY7yR5xud9m-5dMSrVWJOp2vWpBQib6Zpw0gBn9FRGxVGXXPqQgo-ae2sVgFTgbz-8itMZPo-VSQd8eovWcEjLKTI=)]

2.  **Stop the Bleeding:** If the wound is bleeding, apply gentle pressure using a clean cloth.

3.  **Rinse the Wound:** Use clear, running water to rinse the wound. This helps to loosen and remove dirt and debris. You can also use a sterile saline solution to irrigate the wound.[[^2](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYu1LFk0f4bw_7QXkMGxgbzWg_zisfbonQapThnW7JvEU5zJb9wy5NuCaBOukFNVyHGedEZy7Zz1EZQXhxKZfb6xh1_Z5Npc6wQLPiBmH2Io3HoJREIUZz4FxRgsvwsIfD9aSbw3fKJrS7sr0eZoHbP3jmTNF8XeKMCt-tEhndlM1NgmxhPPCTl

In [227]:
test_support = grounding_supports[4]

support_end = test_support.segment.end_index

chunk_indices = test_support.grounding_chunk_indices

ground_dicts = []

for chunk_index in chunk_indices:
    chunk = grounding_chunks[chunk_index]

    ground_dicts.append(
        {
            "cite_index": chunk_index + 1,
            "link": chunk.web.uri,
            "source": chunk.web.title,
        }
    )

In [228]:
ground_dicts

[{'cite_index': 1,
  'link': 'https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYs7nLJ_sEz4uZ3XveeCDOUcpcgvpGJuu1a75zSwZFB2kfYkDOCXn4aEWHjWoUZY7yR5xud9m-5dMSrVWJOp2vWpBQib6Zpw0gBn9FRGxVGXXPqQgo-ae2sVgFTgbz-8itMZPo-VSQd8eovWcEjLKTI=',
  'source': 'webmd.com'},
 {'cite_index': 4,
  'link': 'https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYu4hhFHun-6NFEdWHw7Lh11ARHFmoqyeOzV2BZKWTWZsoPwlnJ7w1Cf71E97d45yds_7DT2orvzwHfvjrcS0QSgmalFk4esGripfee-dbGWvf9FqDUxTiuIEWcoGFUMtD6Ys_aCw8cAFNfX70KzuC48HGc8jxoWlLwfzPUA4atqYjfN',
  'source': 'mayoclinic.org'}]

In [229]:
prefix = full_text[:support_end]
suffix = full_text[support_end:]
# newline_str = collect_trailing_newlines(prefix)
# prefix = prefix.rstrip(newline_str)
print(prefix)

The best way to clean a wound involves several steps:

1.  **Wash Your Hands:** Before you even touch the wound, make sure to wash your hands thoroughly with soap and water or use a hand sanitizer. If possible, wear disposable gloves.

2.  **Stop the Bleeding:** If the wound is bleeding, apply gentle pressure using a clean cloth.

3.  **Rinse the Wound:** Use clear, running water to rinse the wound. This helps to loosen and remove dirt and debris. You can also use a sterile saline solution to irrigate the wound. A syringe can be used to create gentle pressure to help remove debris.

4.  **Clean Around the Wound:** Use a soft washcloth and mild soap to gently clean the skin around the wound. Avoid getting soap directly in the wound, as it can cause irritation.

5.  **Remove Debris:** If there is any dirt or debris remaining in the wound after washing, use tweezers to remove it.


In [230]:
newline_str

'\n\n'

In [231]:
inserted_text = "["

for ground_dict in ground_dicts:
    cite_index = ground_dict["cite_index"]
    link = ground_dict["link"]
    source = ground_dict["source"]
    inserted_text += f"[^{cite_index}]({link}), "

# Remove extra comma and space
inserted_text = inserted_text.rstrip(", ")

inserted_text += f"]"

# print(inserted_text)

updated_text = f"{prefix}{inserted_text}{suffix}"

print(updated_text)

The best way to clean a wound involves several steps:

1.  **Wash Your Hands:** Before you even touch the wound, make sure to wash your hands thoroughly with soap and water or use a hand sanitizer. If possible, wear disposable gloves.

2.  **Stop the Bleeding:** If the wound is bleeding, apply gentle pressure using a clean cloth.

3.  **Rinse the Wound:** Use clear, running water to rinse the wound. This helps to loosen and remove dirt and debris. You can also use a sterile saline solution to irrigate the wound. A syringe can be used to create gentle pressure to help remove debris.

4.  **Clean Around the Wound:** Use a soft washcloth and mild soap to gently clean the skin around the wound. Avoid getting soap directly in the wound, as it can cause irritation.

5.  **Remove Debris:** If there is any dirt or debris remaining in the wound after washing, use tweezers to remove it.[[^1](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYs7nLJ_sEz4uZ3XveeCDOUcpcgvpGJuu1a75z

In [ ]:
updated_text = f"{prefix} ["

for ground_dict in ground_dicts:
    cite_index = ground_dict["cite_index"]
    link = ground_dict["link"]
    source = ground_dict["source"]
    updated_text += f"[^{cite_index}]({link}), "

updated_text = updated_text.rstrip(", ")

updated_text += f"]{newline_str}{suffix}"

print(updated_text)

The best way to clean a wound involves several steps:

1.  **Wash Your Hands:** Before you even touch the wound, make sure to wash your hands thoroughly with soap and water or use a hand sanitizer. If possible, wear disposable gloves.

2.  **Stop the Bleeding:** If the wound is bleeding, apply gentle pressure using a clean cloth.

3.  **Rinse the Wound:** Use clear, running water to rinse the wound. This helps to loosen and remove dirt and debris. You can also use a sterile saline solution to irrigate the wound. A syringe can be used to create gentle pressure to help remove debris.

4.  **Clean Around the Wound:** Use a soft washcloth and mild soap to gently clean the skin around the wound. Avoid getting soap directly in the wound, as it can cause irritation. [[^1](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYs7nLJ_sEz4uZ3XveeCDOUcpcgvpGJuu1a75zSwZFB2kfYkDOCXn4aEWHjWoUZY7yR5xud9m-5dMSrVWJOp2vWpBQib6Zpw0gBn9FRGxVGXXPqQgo-ae2sVgFTgbz-8itMZPo-VSQd8eovWcEjLKTI=), [^

In [225]:
test_support.segment.start_index

771

In [226]:
test_support.segment.end_index

889

In [191]:
for char in reversed(prefix[-10:]):
    if char == "\n":
        print(f"dog")

dog
dog


In [198]:
def collect_trailing_newlines(text):
    """
    Collect the trailing newline characters at the end of the text.

    Parameters:
    text (str): The input text to check for trailing newlines.

    Returns:
    str: Empty string or a string of newline characters.
    """
    new_line_str = ""
    for char in reversed(text):
        if char == "\n":
            new_line_str += char
        else:
            break
    return new_line_str


# Example usage
text = "This is a sample text.\n"

print("This")
print(collect_trailing_newlines(text))
print("That")

This


That


In [181]:
print(f"{full_text[:support_start]}" + "[citation]" + f"{full_text[support_start:]}")

The best way to clean a wound involves several steps:

1.  **Wash Your Hands:** Before you even touch the wound, make sure to wash your hands thoroughly with soap and water or use a hand sanitizer. If possible, wear disposable gloves.

2.  **Stop the Bleeding:** If the wound is bleeding, apply gentle pressure using a clean cloth.

3.  **Rinse the Wound:** Use clear, running water to rinse the wound. This helps to loosen and remove dirt and debris. You can also use a sterile saline solution to irrigate the wound. A syringe can be used to create gentle pressure to help remove debris.

4.  **Clean Around the Wound:** Use a soft washcloth and mild soap to gently clean the skin around the wound. Avoid getting soap directly in the wound, as it can cause irritation.

[citation]5.  **Remove Debris:** If there is any dirt or debris remaining in the wound after washing, use tweezers to remove it. Make sure to clean the tweezers first with isopropyl alcohol. If you can't remove all debris, it's b

In [180]:
print(full_text)

The best way to clean a wound involves several steps:

1.  **Wash Your Hands:** Before you even touch the wound, make sure to wash your hands thoroughly with soap and water or use a hand sanitizer. If possible, wear disposable gloves.

2.  **Stop the Bleeding:** If the wound is bleeding, apply gentle pressure using a clean cloth.

3.  **Rinse the Wound:** Use clear, running water to rinse the wound. This helps to loosen and remove dirt and debris. You can also use a sterile saline solution to irrigate the wound. A syringe can be used to create gentle pressure to help remove debris.

4.  **Clean Around the Wound:** Use a soft washcloth and mild soap to gently clean the skin around the wound. Avoid getting soap directly in the wound, as it can cause irritation.

5.  **Remove Debris:** If there is any dirt or debris remaining in the wound after washing, use tweezers to remove it. Make sure to clean the tweezers first with isopropyl alcohol. If you can't remove all debris, it's best to see

In [169]:
test_support.segment.start_index

771

In [ ]:
full_text

In [164]:
chunk_indices

[0, 3]

In [165]:
grounding_chunks[chunk_indices[0]]

GroundingChunk(retrieved_context=None, web=GroundingChunkWeb(title='webmd.com', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYs7nLJ_sEz4uZ3XveeCDOUcpcgvpGJuu1a75zSwZFB2kfYkDOCXn4aEWHjWoUZY7yR5xud9m-5dMSrVWJOp2vWpBQib6Zpw0gBn9FRGxVGXXPqQgo-ae2sVgFTgbz-8itMZPo-VSQd8eovWcEjLKTI='))

In [151]:
test_support

GroundingSupport(confidence_scores=[0.84713405, 0.81815046], grounding_chunk_indices=[0, 3], segment=Segment(end_index=889, part_index=None, start_index=771, text='5.  **Remove Debris:** If there is any dirt or debris remaining in the wound after washing, use tweezers to remove it.'))

In [152]:
test_chunk

NameError: name 'test_chunk' is not defined

In [140]:
for support in grounding_supports:
    grounding_chunk_indices = support.grounding_chunk_indices

    print(grounding_chunk_indices)
    print(len(grounding_chunk_indices))

[0]
1
[1]
1
[2]
1
[0, 3]
2
[0, 3]
2
[0]
1
[3]
1
[0]
1
[3]
1


In [138]:
annotated_text = full_text

footnotes = []
citation_count = 0
for support in grounding_supports:
    grounding_chunk_indices = support.grounding_chunk_indices

    citation_links = []
    citation_sources = []
    text_to_insert = ""

    for i in grounding_chunk_indices:

        cite_link = ground_chunks[i].web.uri
        cite_source = ground_chunks[i].web.title
        citation_links.append(ground_chunks[i].web.uri)
        citation_sources.append(ground_chunks[i].web.title)

        text_to_insert += f"[[^{i + 1}]({cite_link})] "
        footnotes.append(f"[^{i + 1}]: [{cite_source}]({cite_link})")

    text_to_insert = f"[{text_to_insert}]"

    print(text_to_insert)

    # citation_nums
    # print(ground_chunks[i].web.uri)

[[^1](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYs7nLJ_sEz4uZ3XveeCDOUcpcgvpGJuu1a75zSwZFB2kfYkDOCXn4aEWHjWoUZY7yR5xud9m-5dMSrVWJOp2vWpBQib6Zpw0gBn9FRGxVGXXPqQgo-ae2sVgFTgbz-8itMZPo-VSQd8eovWcEjLKTI=) ]
[[^2](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYu1LFk0f4bw_7QXkMGxgbzWg_zisfbonQapThnW7JvEU5zJb9wy5NuCaBOukFNVyHGedEZy7Zz1EZQXhxKZfb6xh1_Z5Npc6wQLPiBmH2Io3HoJREIUZz4FxRgsvwsIfD9aSbw3fKJrS7sr0eZoHbP3jmTNF8XeKMCt-tEhndlM1NgmxhPPCTlBLqLELeoUGXsLSNXXz62qjGkYgnOW8CwtZhwi49Q27INtk2clXbhu0hsWhai3in91V5VAvar2yhcI6DqP2a9AWjI41Gl4NQ==) ]
[[^3](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYv4ogWt6yP5MQJgQzQY_i6k3hmzPhgwfsDDv5PH7iFlWtTFOxdJ3QoLzt9iGyc1oEwQX89e4AYT46tzeLtFl6lVu7wvlOKlLshUdlD-Fzi31iwRnfJayp1M71PkV-dqGuDMy-D-TuWvpjJBDfiUR2A1rCGH0Ts8sxaBKAaj1bdIhnFbQa5KTnxJhfKwqZwxBw==) ]
[[^4](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYs7nLJ_sEz4uZ3XveeCDOUcpcgvpGJuu1a75zSwZFB2kfYkDOCXn4aEWHjWoUZY7yR5

In [135]:
support  # .segment.end_index

GroundingSupport(confidence_scores=[0.9026266], grounding_chunk_indices=[3], segment=Segment(end_index=1556, part_index=None, start_index=1437, text='It is generally advised to avoid using hydrogen peroxide or iodine directly on the wound, as they can cause irritation.'))

In [126]:
ground_chunks[0].web.uri

GroundingChunk(retrieved_context=None, web=GroundingChunkWeb(title='olympics.com', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYv-PsGYjoXAKPTPUefu6J5Nx5FI70FehGuW0QSrf8ihe1drQhv3GVXoBN8hOqu0f4sPSLkV4-__3yuKyUopbrVXQNfrl8GZo_GZh5OoJrS0F_N_el6e9WbrFHLBUnygtpgkpGkYlZKdq4APOLDC82gOnJ-rp1JoygTzB397J1IsiQCWJ-39dEg9U__qTIEja7CMDiyKv12o0F0='))

In [124]:
ground_chunks[i].web.uri

'https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYv-PsGYjoXAKPTPUefu6J5Nx5FI70FehGuW0QSrf8ihe1drQhv3GVXoBN8hOqu0f4sPSLkV4-__3yuKyUopbrVXQNfrl8GZo_GZh5OoJrS0F_N_el6e9WbrFHLBUnygtpgkpGkYlZKdq4APOLDC82gOnJ-rp1JoygTzB397J1IsiQCWJ-39dEg9U__qTIEja7CMDiyKv12o0F0='

In [120]:
ground_chunk_indices

[0]

In [108]:
def add_footnote_citations(grounding_metadata, full_text):
    """
    Add footnote citations to the full text based on the grounding metadata.
    """

    # Initialize the new annotated text
    annotated_text = full_text

    # Iterate through both the grounding supports and the grounding chunks, and add the footnote citations
    grounding_supports = grounding_metadata.grounding_supports
    ground_chunks = grounding_metadata.grounding_chunks

    footnotes = []
    offset = 0
    for idx, info in enumerate(zip(grounding_supports, ground_chunks), start=1):

        support, chunk = info

        web_uri = chunk.web.uri

        # Update text
        end_index = support.segment.end_index + offset
        citation = f"[[^{idx}]({web_uri})]"
        annotated_text = (
            f"{annotated_text[:end_index]}"
            f"{citation}"
            f"{annotated_text[end_index:]}"
        )

        # Update offset
        offset += len(citation)

        # Add footnote
        title = chunk.web.title
        footnote = f"[{idx}]: [{title}]({web_uri})"
        footnotes.append(footnote)

    # Add the footnotes to the end of the text
    annotated_text += "\n" + "\n".join(footnotes)

    return annotated_text

In [109]:
print(add_footnote_citations(response.candidates[0].grounding_metadata, full_text))

No, the world is not flat. The concept of a flat Earth is an archaic and scientifically disproven idea.[[^1](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYtYLA2MQXxyBHNEI4Qf59QEhFH6QAlUHosTTio-kTCJ5Hn0kzrSyHDa_KJN6-uZ7U46ngqBTkt8Ppf5PVUzSw_tu1I3h-9Ql7524SQfNbwJVtP2KfNkFjM8rnixDfTtMGtiCX0=)] While many ancient cultures believed in a flat Earth, empirical observations have since proven that the Earth is a sphere (more precisely, an ellipsoid).[[^2](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYsuRXLcHW5XLFSjamzkL6eMMopvJ17bCm3n0_6GfnEb8vmCvK2FmG2h-_StK2pdSW8g5pQP3zBtT5cetJh1boGvH3vCc0Ggc_VxoEnVVu24o236X_baeXY1CM0y-PhO7R_PXT3DEXU=)]

[1]: [wikipedia.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYtYLA2MQXxyBHNEI4Qf59QEhFH6QAlUHosTTio-kTCJ5Hn0kzrSyHDa_KJN6-uZ7U46ngqBTkt8Ppf5PVUzSw_tu1I3h-9Ql7524SQfNbwJVtP2KfNkFjM8rnixDfTtMGtiCX0=)
[2]: [britannica.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBn

In [69]:
full_text

"Carlos Alcaraz won the most recent Wimbledon tournament in 2024, defeating Novak Djokovic in the final. It was Alcaraz's second Wimbledon title.\n"

In [70]:
grounding_supports = response.candidates[0].grounding_metadata.grounding_supports
ground_chunks = response.candidates[0].grounding_metadata.grounding_chunks

In [71]:
annotated_text = full_text
annotated_text

"Carlos Alcaraz won the most recent Wimbledon tournament in 2024, defeating Novak Djokovic in the final. It was Alcaraz's second Wimbledon title.\n"

In [73]:
start_index

104

In [74]:
citation

'[^ 2]'

In [76]:
(f"{full_text[:start_index]}" f"{citation}" f"{full_text[start_index:]}")

"Carlos Alcaraz won the most recent Wimbledon tournament in 2024, defeating Novak Djokovic in the final. [^ 2]It was Alcaraz's second Wimbledon title.\n"

In [77]:
full_text

"Carlos Alcaraz won the most recent Wimbledon tournament in 2024, defeating Novak Djokovic in the final. It was Alcaraz's second Wimbledon title.\n"

In [82]:
annotated_text = full_text

footnotes = []
for idx, info in enumerate(zip(grounding_supports, ground_chunks), start=1):

    support, chunk = info

    # Update text
    end_index = support.segment.end_index
    citation = f"[^ {idx}]"
    annotated_text = (
        f"{annotated_text[:end_index]}" f"{citation}" f"{annotated_text[end_index:]}"
    )

    break

    print(annotated_text)
    print("-" * 50)

    # Add footnote
    web_uri = chunk.web.uri
    title = chunk.web.title
    footnote = f"[{idx}]: [{title}]({web_uri})"
    footnotes.append(footnote)

# annotated_text  += "\n\n" + "\n".join(footnotes)

# print(annotated_text)

In [83]:
annotated_text

"Carlos Alcaraz won the most recent Wimbledon tournament in 2024, defeating Novak Djokovic in the final.[^ 1] It was Alcaraz's second Wimbledon title.\n"

In [80]:
start_index

In [81]:
support

GroundingSupport(confidence_scores=[0.85876745], grounding_chunk_indices=[0], segment=Segment(end_index=103, part_index=None, start_index=None, text='Carlos Alcaraz won the most recent Wimbledon tournament in 2024, defeating Novak Djokovic in the final.'))